<a href="https://colab.research.google.com/github/applejxd/colaboratory/blob/master/LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LightGBM で回帰タスクをするサンプル
- [ボストンの住宅価格に関する回帰タスク](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html)
- [説明変数をDataFrame形式で用意](https://scikit-learn.org/stable/datasets/index.html#boston-dataset)

## 前準備

説明変数を DataFrame で用意

In [1]:
import pandas as pd
from sklearn.datasets import load_boston

data = load_boston()
train_x: pd.DataFrame = pd.DataFrame(data.data, columns = data.feature_names)

print(len(train_x))
display(train_x.head())

506


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


目的変数も DataFrame で用意

In [2]:
train_y: pd.DataFrame = pd.DataFrame(data.target, columns = ["MEDV"])
display(train_y.head())

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


学習を行ってモデルを戻す関数を定義

In [3]:
from typing import Dict
# !pip install optuna
# import optuna.integration.lightgbm as lgb
import lightgbm as lgb

def Fit(tr_x: pd.DataFrame, tr_y: pd.Series,
        va_x: pd.DataFrame, va_y: pd.Series,
        params: Dict = {}) -> lgb.Booster:
    """
    :param tr_x: 学習用説明変数
    :param tr_y: 学習用目的変数
    :param va_x: バリデーション用説明変数
    :param va_y: バリデーション用目的変数
    :return: 学習済みモデル
    """
    # LightGBM が扱う形式に変換
    lgb_train: lgb.Dataset = lgb.Dataset(tr_x, tr_y)
    lgb_eval: lgb.Dataset = lgb.Dataset(va_x, va_y)

    # ハイパーパラメータ
    if len(params) == 0:
        params = {
            'objective': 'regression_l1', 'metrics': 'l1', 
            'seed': 71, 'learning_rate': 0.1
        }

    # 学習の実施（Early Stopping）
    booster = lgb.train(params, lgb_train, verbose_eval=False,
                        num_boost_round=1000, early_stopping_rounds=50,
                        valid_names=['train', 'valid'],
                        valid_sets=[lgb_train, lgb_eval])
    return booster

クロスバリデーションの準備

In [4]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=4, shuffle=True, random_state=71)

## 学習の実施

学習の実施

In [5]:
# 学習・バリデーションデータの分離
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# 学習
model: lgb.Booster = Fit(tr_x, tr_y, va_x, va_y)

# 特徴量評価
importance = pd.DataFrame(model.feature_importance(),
                          index=tr_x.columns.values, 
                          columns=["importance"])
display(importance)

,importance
CRIM,854
ZN,87
INDUS,260
CHAS,24
NOX,561
RM,1011
AGE,967
DIS,820
RAD,122
TAX,298


バリデーションの実施

In [6]:
import numpy as np
from sklearn.metrics import mean_absolute_error

va_pred: np.ndarray = model.predict(va_x)
score: float = mean_absolute_error(va_y, va_pred)
print(f'MAE: {score:.4f}')

MAE: 2.1544


## クロスバリデーションとハイパーパラメータの調整

クロスバリデーションを行う関数を定義

In [7]:
from hyperopt import STATUS_OK

def score(params):
    scores = []
    for tr_idx, va_idx in kf.split(train_x):
        # 学習・バリデーションデータの分離
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

        # 学習
        model = Fit(tr_x, tr_y, va_x, va_y, params)

        # バリデーション
        va_pred = model.predict(va_x)
        score = mean_absolute_error(va_y, va_pred)
        scores.append(score)
        
    # 情報を記録しておく
    score_ave = np.mean(scores)
    history.append((params, score_ave))

    return {'loss': score_ave, 'status': STATUS_OK}

探索するパラメータの空間を指定

cf. [Laurae++: PARAMETERS](https://sites.google.com/view/lauraepp/parameters)

In [8]:
from hyperopt import hp

space = {
    'objective': 'regression_l1', 'metric': 'l1',
    'seed': 71, 'learning_rate': 0.1,
    'lamda_l1': hp.loguniform('lamda_l1', -8, 2),
    'lamda_l2': hp.loguniform('lamda_l2', -8, 2),
    'num_leaves': hp.choice('num_leaves', np.arange(2, 256+1, dtype=int)),
    'feature_fraction': hp.uniform('feature_fraction', 0.4, 1.0),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.4, 1.0),
    'bagging_freq': hp.choice('bagging_freq', np.arange(1, 7+1, dtype=int)),
    'min_child_samples': hp.choice('min_child_samples', np.arange(5, 100+1, dtype=int)),
}

hyperoptによるパラメータ探索の実行

In [9]:
from hyperopt import fmin, tpe, Trials

trials = Trials()
history = []
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=100)

100%|██████████| 100/100 [01:41<00:00,  1.02s/it, best loss: 2.171113322803114]


{'bagging_fraction': 0.9710906593670214,
 'bagging_freq': 6,
 'feature_fraction': 0.636688416939736,
 'lamda_l1': 0.5779140900210183,
 'lamda_l2': 0.18533065804254428,
 'min_child_samples': 7,
 'num_leaves': 57}

記録した情報からスコアを出力

In [10]:
history = sorted(history, key=lambda tpl: tpl[1])
# 最新の履歴
best = history[0]
print(f'params: {best[0]}')
print(f'score: {best[1]:.4f}')

params: {'bagging_fraction': 0.9710906593670214, 'bagging_freq': 7, 'feature_fraction': 0.636688416939736, 'lamda_l1': 0.5779140900210183, 'lamda_l2': 0.18533065804254428, 'learning_rate': 0.1, 'metric': 'l1', 'min_child_samples': 12, 'num_leaves': 59, 'objective': 'regression_l1', 'seed': 71}
score: 2.1711
